In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import compress

## Load the datasets and review

In [2]:
miami_2014 = pd.read_csv('data/miami_dade_acs5-2014_census.csv')
miami_2014 = miami_2014.set_index("index")

miami_2019 = pd.read_csv('data/miami_dade_acs5-2019_census.csv')
miami_2019 = miami_2019.set_index("index")

data_dic = pd.read_csv('data/acs5_variable_dict_2014_2019.csv')

In [3]:
data_dic

,variable_code,label,concept,predicateType,group,limit,predicateOnly,acs_year
0,DP02_0019EA,Annotation of Estimate!!RELATIONSHIP!!Populati...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0,True,2014
1,DP02_0126E,Estimate!!ANCESTRY!!Total population!!Danish,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,True,2014
2,DP02_0072EA,Annotation of Estimate!!DISABILITY STATUS OF T...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0,True,2014
3,DP02_0069PMA,Annotation of Percent Margin of Error!!VETERAN...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0,True,2014
4,DP02_0126M,Margin of Error!!ANCESTRY!!Total population!!D...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,True,2014
5,DP02_0057PMA,Annotation of Percent Margin of Error!!SCHOOL ...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0,True,2014
6,DP02_0125M,Margin of Error!!ANCESTRY!!Total population!!C...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,True,2014
7,DP02_0054PM,Percent Margin of Error!!SCHOOL ENROLLMENT!!Po...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,float,DP02,0,True,2014
8,DP02_0041MA,Annotation of Margin of Error!!FERTILITY!!Numb...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0,True,2014
9,DP02_0070PMA,Annotation of Percent Margin of Error!!DISABIL...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0,True,2014


In [4]:
miami_2014

,DP02_0001E,DP02_0001PE,DP02_0002E,DP02_0002PE,DP02_0003E,DP02_0003PE,DP02_0004E,DP02_0004PE,DP02_0005E,DP02_0005PE,...,B25087_027E,B25087_028E,B25087_029E,B25088_001E,B25088_002E,B25088_003E,B25092_001E,B25092_002E,B25092_003E,GEOID
index,,,,,,,,,,,,,,,,,,,,,
"Census Tract 1.07, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000107",1255,1255,611,48.7,171,13.6,468,37.3,142,11.3,...,0,0,284,1624,2547,1001,24.2,24.5,22.3,12086000107
"Census Tract 1.18, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000118",506,506,332,65.6,90,17.8,229,45.3,27,5.3,...,0,11,219,2185,3388,1001,46.5,50.0,26.3,12086000118
"Census Tract 1.19, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000119",2206,2206,1192,54.0,498,22.6,894,40.5,249,11.3,...,45,73,552,1146,1672,828,26.3,38.7,16.0,12086000119
"Census Tract 1.21, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000121",780,780,487,62.4,100,12.8,401,51.4,70,9.0,...,0,0,421,1906,4001,1001,36.0,25.8,41.5,12086000121
"Census Tract 1.20, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000120",1855,1855,998,53.8,475,25.6,789,42.5,403,21.7,...,40,41,280,1696,1953,1001,28.2,33.8,13.1,12086000120
"Census Tract 1.23, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000123",2177,2177,1061,48.7,248,11.4,845,38.8,185,8.5,...,167,164,321,999,2246,624,34.3,50.0,26.5,12086000123
"Census Tract 1.24, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000124",1065,1065,545,51.2,180,16.9,240,22.5,67,6.3,...,8,15,0,1094,1255,446,46.0,49.8,25.2,12086000124
"Census Tract 1.25, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000125",1861,1861,969,52.1,270,14.5,573,30.8,134,7.2,...,45,97,414,898,1713,751,31.5,45.2,26.6,12086000125
"Census Tract 1.26, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:000126",1282,1282,592,46.2,229,17.9,560,43.7,206,16.1,...,89,35,269,823,1238,676,40.8,50.0,39.5,12086000126


In [5]:
len(miami_2014)

519

In [6]:
len(miami_2019)

519

## Find matching and unique columns between both datasets

In [7]:
# Identify all missmatched columns
mismatched_columns = list(set(miami_2014.columns).symmetric_difference(set(miami_2019.columns)))
len(mismatched_columns)

64

In [8]:
# Identify all matching columns
matched_columns = list(set(miami_2014.columns) & set(miami_2019.columns))
len(matched_columns)

1551

## Statistical testing between datasets

In [9]:
# Perform student's t test
t_check=stats.ttest_rel(miami_2019[matched_columns], miami_2014[matched_columns])
t_check
alpha=0.05

statistically_similar = list(compress(matched_columns, (t_check.pvalue > alpha)))
statistically_different = list(compress(matched_columns, (t_check.pvalue <= alpha)))

c:\anaconda3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\anaconda3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\anaconda3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  
c:\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less_equal
  import sys


In [10]:
print("Statistically Similar Columns: {}".format(len(statistically_similar)))
print("Statistically Different Columns: {}".format(len(statistically_different)))

Statistically Similar Columns: 634
Statistically Different Columns: 883


In [11]:
correlations = pd.DataFrame(columns = ['Variable', 'Correlation', 'PValue'])

for variable in matched_columns:
    # Using spearman as most columns do not appear to follow normal distribution
    corr = stats.spearmanr(miami_2014[variable], miami_2019[variable], axis=0)
    correlations = correlations.append({
        'Variable': variable,
        'Correlation': 0 if np.isnan(corr.correlation) else corr.correlation,
        'PValue': 0 if np.isnan(corr.pvalue) else corr.pvalue,
        'NullHypothesisRejected': True if corr.pvalue <= 0.05 else False
    }, ignore_index = True)
    
correlations.sort_values(by='Correlation', ascending=False, inplace=True)

correlations

c:\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\anaconda3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\anaconda3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\anaconda3\envs\py36\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,Variable,Correlation,PValue,NullHypothesisRejected
987,S2502_C03_009E,0.153788,0.000438,1.0
819,DP05_0037PE,0.152968,0.000470,1.0
374,S2502_C02_009E,0.151320,0.000542,1.0
918,S2502_C01_009E,0.146300,0.000829,1.0
1307,DP04_0052E,0.143578,0.001038,1.0
37,S2502_C01_024E,0.136507,0.001827,1.0
1171,S2506_C01_038E,0.134850,0.002079,1.0
1002,S2504_C01_011E,0.127969,0.003497,1.0
1321,DP04_0092PE,0.120583,0.005951,1.0
1259,DP04_0052PE,0.119980,0.006207,1.0


## Reviewing difference between same variable code in both datasets

In [12]:
data_dic[data_dic.variable_code == "DP04_0005E"]

,variable_code,label,concept,predicateType,group,limit,predicateOnly,acs_year
2938,DP04_0005E,Estimate!!HOUSING OCCUPANCY!!Rental vacancy rate,SELECTED HOUSING CHARACTERISTICS,float,DP04,0,True,2014
7043,DP04_0005E,Estimate!!HOUSING OCCUPANCY!!Total housing uni...,SELECTED HOUSING CHARACTERISTICS,float,DP04,0,True,2019


In [13]:
miami_2014["DP04_0037E"].describe()

count     519.000000
mean     1916.369942
std       867.580485
min         0.000000
25%      1395.000000
50%      1841.000000
75%      2302.000000
max      6800.000000
Name: DP04_0037E, dtype: float64

In [14]:
miami_2019["DP04_0037E"].describe()

count    5.190000e+02
mean    -1.669878e+07
std      1.042815e+08
min     -6.666667e+08
25%      3.900000e+00
50%      4.600000e+00
75%      5.400000e+00
max      1.000000e+01
Name: DP04_0037E, dtype: float64

## Calculate percentage of change from 2014 to 2019

In [15]:
percentage_diff = (
    (miami_2019[matched_columns] - miami_2014[matched_columns]) 
    / miami_2014[matched_columns]) * 100

# Replace infinite values with nan
percentage_diff.replace([np.inf, -np.inf], np.nan, inplace=True)

# Export results to CSV
percentage_diff.to_csv('out/percentage_differnces_miami.csv')

In [16]:
percentage_diff["DP04_0005E"].describe()

count     384.000000
mean        1.248603
std       236.034105
min      -100.000015
25%       -86.382897
50%       -42.543860
75%        11.612319
max      3262.500000
Name: DP04_0005E, dtype: float64

In [17]:
percentage_diff["DP04_0005E"].argmax()

c:\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


'Census Tract 67.11, Miami-Dade County, Florida: Summary level: 140, state:12> county:086> tract:006711'

In [18]:
# Calculate zscores for all variable percentage means
zscores = stats.zscore(list(percentage_diff.mean(axis=0, skipna=True)), nan_policy='omit')

In [19]:
# Identify outliers using zscores
list(compress(matched_columns, (np.abs(zscores) > 3)))

c:\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


['DP05_0004PE', 'DP04_0125PE', 'DP04_0109PE', 'DP04_0135PE']

In [20]:
percentage_diff["DP04_0005E"].plot(kind="box", figsize=(10,20))

In [21]:
percentage_diff.describe()

,DP03_0070PE,B25063_002E,DP04_0020PE,DP03_0031E,DP02_0062E,DP04_0013PE,B25074_035E,B25074_059E,B25074_026E,S2506_C01_048E,...,DP03_0064E,DP03_0105PE,DP04_0027E,DP04_0030PE,DP05_0024E,DP05_0073E,DP05_0060PE,S2504_C03_032E,DP03_0092PE,S1701_C01_043E
count,502.000000,508.000000,504.000000,501.000000,508.000000,441.000000,296.000000,57.000000,341.000000,35.000000,...,508.000000,519.000000,363.000000,516.000000,509.000000,460.000000,509.000000,92.000000,519.0,505.000000
mean,31.805902,33.346406,103.970838,76.120141,100.686256,20.092168,39.030743,53.190265,-0.928807,2814.609774,...,16.783957,26.364692,6999.267326,-16.933977,-58.502646,371.429267,-93.187819,972.962430,0.0,1164.675287
std,141.981367,279.078882,835.304476,277.573458,133.920196,134.767462,205.728549,270.943894,134.055129,4719.608077,...,187.263062,26.799273,8702.484074,70.246875,25.770135,1380.297772,30.568223,1974.398468,0.0,1680.995003
min,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000015,-100.000000,-100.000000,-100.000000,-100.000000,...,-100.000000,-23.172628,163.461538,-100.000015,-100.000000,-100.000000,-100.000000,-100.000000,-0.0,-41.493268
25%,-32.056604,-4.820598,-65.214580,-9.697933,14.934169,-15.686275,-100.000000,-100.000000,-100.000000,-100.000000,...,-3.945612,7.183257,1646.511848,-61.634615,-68.961303,-85.167778,-100.000000,-100.000000,-0.0,399.196787
50%,3.740672,13.786316,-23.294574,23.000000,68.361674,0.823045,-28.919355,-66.666667,-38.636364,66.666667,...,5.088602,20.411392,3914.285714,-31.563701,-61.055578,-26.713648,-100.000000,161.278195,-0.0,887.500000
75%,45.371397,35.583024,41.748768,82.183908,159.984956,23.456790,88.398693,88.888889,32.558140,5300.000000,...,16.877826,37.265536,9126.190476,3.664027,-51.428571,193.593544,-100.000000,1237.316176,-0.0,1397.354497
max,1700.000000,6185.714286,12366.666667,4700.000000,1227.956989,2012.500000,1650.000000,1500.000000,616.666667,15500.000000,...,4200.000000,153.453453,93200.000000,461.956522,425.581395,18200.000000,400.000000,11284.615385,-0.0,28200.000000


## To be continued pending completion of data dictionary cleanup